In [ ]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

# 혼방율 데이터 로드
cleaned_fiber_data_path = '/path'
cleaned_fiber_data = pd.read_csv(cleaned_fiber_data_path)

# 첫 번째 열을 "Sample Number" 로 수정
cleaned_fiber_data.rename(columns={cleaned_fiber_data.columns[0]: 'Sample Number'}, inplace=True)
cleaned_fiber_data.set_index('Sample Number', inplace=True)

# 설문 조사 결과 로드
folder_path = '/path'
survey_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
survey_data = pd.DataFrame()

for file in survey_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    # 첫 번째 열의 이름을 'Sample Number'로 지정하고 인덱스로 설정
    df.rename(columns={df.columns[0]: 'Sample Number'}, inplace=True)
    df.set_index('Sample Number', inplace=True)
    survey_data = pd.concat([survey_data, df])

# 'Sample Number'를 기준으로 혼방율 데이터와 설문 데이터 병합
merged_data = pd.merge(cleaned_fiber_data, survey_data, left_index=True, right_index=True)

# 모든 데이터에 대한 상관 행렬 계산
correlation_matrix_pearson = merged_data.corr(method='pearson')
correlation_matrix_spearman = merged_data.corr(method='spearman')
correlation_matrix_kendall = merged_data.corr(method='kendall')

# 상관 행렬의 관련 부분 선택
fabric_columns = cleaned_fiber_data.columns
attributes = ['smoothness', 'softness', 'thickness', 'flexibility']
correlation_pearson = correlation_matrix_pearson.loc[fabric_columns, attributes]
correlation_spearman = correlation_matrix_spearman.loc[fabric_columns, attributes]
correlation_kendall = correlation_matrix_kendall.loc[fabric_columns, attributes]

# 상관 결과를 CSV 파일로 저장하여 추가 분석
correlation_pearson.to_csv('/Users/Keunsu/Desktop/EDA/TextureSurvey/Correlation_All_Data_Results_Pearson.csv', index=True)
correlation_spearman.to_csv('/Users/Keunsu/Desktop/EDA/TextureSurvey/Correlation_All_Data_Results_Spearman.csv', index=True)
correlation_kendall.to_csv('/Users/Keunsu/Desktop/EDA/TextureSurvey/Correlation_All_Data_Results_Kendall.csv', index=True)

print("상관 결과가 CSV 파일로 저장되었습니다.")

# 상관 행렬을 사용하여 속성의 스케일된 값을 계산하는 함수
def calculate_scaled_values_from_correlation(attribute, correlation_matrix, cleaned_fiber_data):
    # 특정 속성에 대한 상관 관계 가져오기
    correlation_attribute = correlation_matrix[attribute]
    
    # 중요한 상관 관계 식별
    strongest_correlations = correlation_attribute.abs().sort_values(ascending=False)
    
    # 각 샘플 번호에 대한 속성 평균 값 계산
    mean_attribute = survey_data.groupby(survey_data.index).mean()[attribute]
    
    # 혼방율 데이터와 병합
    merged_data = pd.merge(cleaned_fiber_data, mean_attribute, left_index=True, right_index=True)
    
    # 가중 속성 값 계산
    weights = strongest_correlations / strongest_correlations.sum()
    weighted_attribute = (merged_data[strongest_correlations.index] * weights).sum(axis=1)
    
    # 가중 속성 값을 1에서 5 범위로 정규화
    min_attribute = weighted_attribute.min()
    max_attribute = weighted_attribute.max()
    scaled_attribute = 1 + (weighted_attribute - min_attribute) * 4 / (max_attribute - min_attribute)
    
    return scaled_attribute

# Pearson, Spearman, Kendall Tau 상관 관계를 사용하여 각 속성의 스케일된 값 계산
scaled_values_pearson = {}
scaled_values_spearman = {}
scaled_values_kendall = {}

for attribute in attributes:
    scaled_values_pearson[attribute] = calculate_scaled_values_from_correlation(attribute, correlation_pearson, cleaned_fiber_data)
    scaled_values_spearman[attribute] = calculate_scaled_values_from_correlation(attribute, correlation_spearman, cleaned_fiber_data)
    scaled_values_kendall[attribute] = calculate_scaled_values_from_correlation(attribute, correlation_kendall, cleaned_fiber_data)

# 스케일된 값에 대한 데이터프레임 생성
scaled_values_pearson_df = pd.DataFrame(scaled_values_pearson)
scaled_values_spearman_df = pd.DataFrame(scaled_values_spearman)
scaled_values_kendall_df = pd.DataFrame(scaled_values_kendall)

# 결합된 데이터프레임 출력
pd.set_option('display.max_rows', None)  # 모든 행을 표시하도록 설정
print("각 속성에 대한 스케일된 값 (Pearson 상관 관계):")
print(scaled_values_pearson_df)

print("각 속성에 대한 스케일된 값 (Spearman 상관 관계):")
print(scaled_values_spearman_df)

print("각 속성에 대한 스케일된 값 (Kendall Tau 상관 관계):")
print(scaled_values_kendall_df)

# 결합된 데이터프레임을 CSV 파일로 저장하여 추가 분석
scaled_values_pearson_df.to_csv('/Users/Keunsu/Desktop/EDA/TextureSurvey/Scaled_Values_Pearson.csv', index=True)
scaled_values_spearman_df.to_csv('/Users/Keunsu/Desktop/EDA/TextureSurvey/Scaled_Values_Spearman.csv', index=True)
scaled_values_kendall_df.to_csv('/Users/Keunsu/Desktop/EDA/TextureSurvey/Scaled_Values_Kendall.csv', index=True)

print("스케일된 값이 CSV 파일로 저장되었습니다.")

# 각 속성에 대한 스케일된 값을 시각화
for attribute in attributes:
    plt.figure(figsize=(14, 8))
    plt.plot(scaled_values_pearson_df.index, scaled_values_pearson_df[attribute], label='Pearson', color='blue', marker='o')
    plt.plot(scaled_values_spearman_df.index, scaled_values_spearman_df[attribute], label='Spearman', color='green', marker='x')
    plt.plot(scaled_values_kendall_df.index, scaled_values_kendall_df[attribute], label='Kendall Tau', color='red', marker='s')
    plt.title(f'{attribute.capitalize()} 각 샘플의 평가 (스케일 1-5)')
    plt.xlabel('샘플 번호')
    plt.ylabel(f'스케일된 {attribute.capitalize()}')
    plt.ylim(1, 5)  # y축을 1에서 5로 스케일링
    plt.xticks(rotation=90)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout() 
    plt.show()
